In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection 
db_username = "root"
db_password = " "
db_host = "localhost"
db_name = "hackathon"

# Creating connection
engine = create_engine(f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}")

# Fetching data from SQL table
query = "SELECT * FROM sales_data"  
df = pd.read_sql(query, engine)


print(df.head())


In [ ]:
# Check for any missed missing values
print(df.isnull().sum())

# Display cleaned data
print(df.info())


In [ ]:
#EDA
#Total Sales by Category
import matplotlib.pyplot as plt
import seaborn as sns

# Group by category
sales_by_category = df.groupby('Category')['TotalPrice'].sum()

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(x=sales_by_category.index, y=sales_by_category.values)
plt.xticks(rotation=45)
plt.title("Total Sales by Category")
plt.xlabel("Category")
plt.ylabel("Total Sales")
plt.show()


In [ ]:
#Sales trend overtime
# Convert OrderDate to datetime format
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Aggregate total sales by date
sales_trend = df.groupby('OrderDate')['TotalPrice'].sum()

# Plot sales trend
plt.figure(figsize=(12, 5))
sales_trend.plot()
plt.title("Daily Sales Trend")
plt.xlabel("Date")
plt.ylabel("Total Sales")
plt.grid()
plt.show()


In [ ]:
#Gender-Based Spending Trends
# Average spending per gender
gender_spending = df.groupby('Gender')['TotalPrice'].mean().round(2)

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(x=gender_spending.index, y=gender_spending.values, palette="coolwarm")
plt.title("Average Spending per Customer by Gender")
plt.xlabel("Gender")
plt.ylabel("Average Total Price")
plt.show()


In [ ]:
#The Best-Selling Products
# Top-selling products by revenue
top_products = df.groupby('ProductName')['TotalPrice'].sum().sort_values(ascending=False).head(5)

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(x=top_products.index, y=top_products.values, palette="viridis")
plt.xticks(rotation=45)
plt.title("Top 5 Best-Selling Products")
plt.xlabel("Product Name")
plt.ylabel("Total Sales")
plt.show()


In [ ]:
#Monthly Sales Trends
# Convert OrderDate to datetime format
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Resample data to get monthly total sales
monthly_sales = df.resample('ME', on='OrderDate')['TotalPrice'].sum()

# Plot
plt.figure(figsize=(12, 5))
monthly_sales.plot(marker='o', linestyle="-", color="purple")
plt.title("Monthly Sales Trend")
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.grid()
plt.show()


In [ ]:
#Revenue by Region
# Sales by Region
sales_by_region = df.groupby('Region')['TotalPrice'].sum().sort_values(ascending=False)

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(x=sales_by_region.index, y=sales_by_region.values, palette="magma")
plt.xticks(rotation=45)
plt.title("Sales Distribution by Region")
plt.xlabel("Region")
plt.ylabel("Total Sales")
plt.show()


In [ ]:
#Order Delivery Status
# Count shipping statuses
shipping_status_counts = df['ShippingStatus'].value_counts()

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(x=shipping_status_counts.index, y=shipping_status_counts.values, palette="coolwarm")
plt.title("Shipping Status Distribution")
plt.xlabel("Shipping Status")
plt.ylabel("Number of Orders")
plt.show()


In [ ]:
# Dashboard using streamlit

import streamlit as st


# Load dataset with error handling
def load_data():
    try:
        df = pd.read_csv(r"C:\Hackathon\-E-commerce-Sales-Analysis-Hackathon\sales_data.csv")  
        if 'OrderDate' in df.columns:
            df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')
        return df
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return pd.DataFrame()

df = load_data()

if df.empty:
    st.stop()

# Sidebar Filters
st.sidebar.header("Filter Data")
if 'Category' in df.columns:
    selected_category = st.sidebar.multiselect("Select Category", df['Category'].dropna().unique(), key="category_filter")
else:
    selected_category = []

if 'Region' in df.columns:
    selected_region = st.sidebar.multiselect("Select Region", df['Region'].dropna().unique(), key="region_filter")
else:
    selected_region = []

# Apply filters
if selected_category:
    df = df[df['Category'].isin(selected_category)]
if selected_region:
    df = df[df['Region'].isin(selected_region)]

# Main Dashboard
st.title("📊 E-Commerce Sales Dashboard")

# Sales Overview
if 'TotalPrice' in df.columns:
    st.subheader("💰 Sales Overview")
    total_sales = df['TotalPrice'].sum()
    avg_order_value = df['TotalPrice'].mean()
    st.metric("Total Sales", f"${total_sales:,.2f}")
    st.metric("Average Order Value", f"${avg_order_value:,.2f}")

# Top-Selling Products
if 'ProductName' in df.columns:
    st.subheader("🏆 Top 5 Best-Selling Products")
    top_products = df.groupby('ProductName')['TotalPrice'].sum().sort_values(ascending=False).head(5)
    st.bar_chart(top_products)

# Monthly Sales Trend
if 'OrderDate' in df.columns and 'TotalPrice' in df.columns:
    st.subheader("📈 Monthly Sales Trend")
    monthly_sales = df.resample('ME', on='OrderDate')['TotalPrice'].sum()
    monthly_sales.index = monthly_sales.index.strftime('%Y-%m')
    st.line_chart(monthly_sales)

# Customer Segmentation
if 'Gender' in df.columns:
    st.subheader("👥 Customer Spending by Gender")
    gender_spending = df.groupby('Gender')['TotalPrice'].mean()
    fig, ax = plt.subplots()
    sns.barplot(x=gender_spending.index, y=gender_spending.values, ax=ax, palette="coolwarm")
    ax.set_title("Average Spending per Customer by Gender")
    ax.set_xlabel("Gender")
    ax.set_ylabel("Average Total Price")
    st.pyplot(fig)

# Shipping Performance
if 'ShippingStatus' in df.columns:
    st.subheader("🚚 Shipping Performance")
    shipping_status_counts = df['ShippingStatus'].value_counts()
    fig, ax = plt.subplots()
    sns.barplot(x=shipping_status_counts.index, y=shipping_status_counts.values, ax=ax, palette="viridis")
    ax.set_title("Shipping Performance")
    ax.set_xlabel("Shipping Status")
    ax.set_ylabel("Number of Orders")
    st.pyplot(fig)

# Insights
st.write("### 📌 Insights")
st.write("- Focus on best-selling products to boost sales.")
st.write("- Identify top-performing regions for marketing campaigns.")
st.write("- Monitor seasonal trends for better inventory planning.")
st.write("- Improve shipping times to enhance customer satisfaction.")

st.write("---")
st.write("🔍 _Explore the data dynamically using the filters in the sidebar._")
